## Import libraries

In [2]:
import numpy as np   # import numpy
import pandas as pd  # import pandas
import os
import gc   # for gabage collection
import seaborn as sns  # data visualization lib
import matplotlib.pyplot as plt
import glob 
import pry
from scipy import stats
%matplotlib inline

<hr  style= "
  display: block; 
  margin-top: 0.5em; 
  margin-bottom: 0.5em;
  margin-left: auto;
  margin-right: auto;
  border-style: inset;
  border-width: 15px "/>

## Create a few algos for viz so that they can be called anytime


In [2]:
# some customized function for plotting data
def plot_corr(df,cols):
    fig = plt.figure(figsize=(10,8))
    sns.heatmap(df[cols].corr(),cmap='RdBu_r', annot=True, center=0.0)
    plt.show()
    
def plot_count(df,col,fsize,rotation=None,fillna=True):
    fig = plt.figure(figsize=fsize)
    if fillna:
        value_count = df[col].fillna('unknown').value_counts()
    sns.barplot(value_count.index,value_count.values)
    plt.xticks(rotation=rotation)
    plt.title('value counts for {}'.format(col))
    plt.show()

<hr  style= "
  display: block; 
  margin-top: 0.5em; 
  margin-bottom: 0.5em;
  margin-left: auto;
  margin-right: auto;
  border-style: inset;
  border-width: 15px "/>

## Load All Data

In [ ]:
def load_data(path): 
    return glob.glob(path+"*.csv")   

In [ ]:
df_files = load_data('/Users/mega/concordia_courses/1260_ML/home_credit_risk/data/')
df_files

In [ ]:
df_names = []
for i in range(len(df_files)):
    df_names.append('df' + str(i))
print(df_names)



In [ ]:
all_df = {'df0': df_files[0], 'df1': df_files[1], 'df2': df_files[2], 'df3': df_files[3], 'df4': df_files[4], 'df5': df_files[5], 'df6': df_files[6]}
all_df_list = []
for key, value in all_df.items():
    key = pd.read_csv(value)
    all_df_list.append(key)
df_pos_cash, df_cc_bal, df_inst_pay, df_app_train, df_burr, df5_prev_app, df_burr_bal = all_df_list[0], all_df_list[1],  all_df_list[2],  all_df_list[3], all_df_list[4],  all_df_list[5],  all_df_list[6]



<hr  style= "
  display: block; 
  margin-top: 0.5em; 
  margin-bottom: 0.5em;
  margin-left: auto;
  margin-right: auto;
  border-style: inset;
  border-width: 15px "/>

## Load Single file

In [3]:
df_app_train = pd.read_csv('/Users/mega/concordia_courses/1260_ML/home_credit_risk/data/application_train.csv')

<hr  style= "
  display: block; 
  margin-top: 0.5em; 
  margin-bottom: 0.5em;
  margin-left: auto;
  margin-right: auto;
  border-style: inset;
  border-width: 15px "/>

## Memory Stats

NameError: name 'total_mem_use' is not defined

### Calculate total memory before forcing data types for all dataframes


In [5]:
total_mem_use = []
for i in all_df_list:
  total_mem_use.append(i.memory_usage().sum())
print('Total memory usage of dataframes df0 through df6 are {:.2f} MB'.format(sum(total_mem_use)/1024**2))

NameError: name 'all_df_list' is not defined

### Calculate total memory before forcing data types for single dataframe

In [7]:
total_mem_use = []
total_mem_use.append(df_app_train.memory_usage().sum())
print('Total memory usage of df_app_train is {:.2f} MB'.format(sum(total_mem_use)/1024**2))

Total memory usage of df_app_train is 286.23 MB


<hr  style= "
  display: block; 
  margin-top: 0.5em; 
  margin-bottom: 0.5em;
  margin-left: auto;
  margin-right: auto;
  border-style: inset;
  border-width: 15px "/>

## Data Preprocessing

In [8]:
df_app_train.shape

(307511, 122)

### Create a dataframe for all missing values with column names and nan count.

In [9]:
nan_info = pd.DataFrame(df_app_train.isnull().sum()).reset_index()
nan_info.columns = ['col','nan_cnt']
nan_info[nan_info['nan_cnt'] !=0]



,col,nan_cnt
9,AMT_ANNUITY,12
10,AMT_GOODS_PRICE,278
11,NAME_TYPE_SUITE,1292
21,OWN_CAR_AGE,202929
28,OCCUPATION_TYPE,96391
...,...,...
117,AMT_REQ_CREDIT_BUREAU_DAY,41519
118,AMT_REQ_CREDIT_BUREAU_WEEK,41519
119,AMT_REQ_CREDIT_BUREAU_MON,41519
120,AMT_REQ_CREDIT_BUREAU_QRT,41519


### Extract all columns names with missing values. So we can itterate.

In [10]:
cols_with_missing = nan_info.loc[nan_info.nan_cnt>0].col.values
cols_with_missing 


array(['AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'OWN_CAR_AGE',
       'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'EXT_SOURCE_1',
       'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG',
       'BASEMENTAREA_AVG', 'YEARS_BEGINEXPLUATATION_AVG',
       'YEARS_BUILD_AVG', 'COMMONAREA_AVG', 'ELEVATORS_AVG',
       'ENTRANCES_AVG', 'FLOORSMAX_AVG', 'FLOORSMIN_AVG', 'LANDAREA_AVG',
       'LIVINGAPARTMENTS_AVG', 'LIVINGAREA_AVG',
       'NONLIVINGAPARTMENTS_AVG', 'NONLIVINGAREA_AVG', 'APARTMENTS_MODE',
       'BASEMENTAREA_MODE', 'YEARS_BEGINEXPLUATATION_MODE',
       'YEARS_BUILD_MODE', 'COMMONAREA_MODE', 'ELEVATORS_MODE',
       'ENTRANCES_MODE', 'FLOORSMAX_MODE', 'FLOORSMIN_MODE',
       'LANDAREA_MODE', 'LIVINGAPARTMENTS_MODE', 'LIVINGAREA_MODE',
       'NONLIVINGAPARTMENTS_MODE', 'NONLIVINGAREA_MODE',
       'APARTMENTS_MEDI', 'BASEMENTAREA_MEDI',
       'YEARS_BEGINEXPLUATATION_MEDI', 'YEARS_BUILD_MEDI',
       'COMMONAREA_MEDI', 'ELEVATORS_MEDI', 'ENTRANCES_MEDI',
       'FLOOR

### Create a df with missing values

In [11]:
df_app_train_missing = pd.DataFrame(df_app_train[cols_with_missing])
df_app_train_missing.shape

(307511, 67)

### Remove columns with missing values from orignal df

In [12]:
df_app_train.drop(list(cols_with_missing), axis = 1, inplace=True )
df_app_train.shape

(307511, 55)

### Create a test df for testing

In [13]:
df_test = df_app_train_missing.copy()
df_test.shape

(307511, 67)

In [31]:
df_app_train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 55 columns):
SK_ID_CURR                     307511 non-null int64
TARGET                         307511 non-null int64
NAME_CONTRACT_TYPE             307511 non-null object
CODE_GENDER                    307511 non-null object
FLAG_OWN_CAR                   307511 non-null object
FLAG_OWN_REALTY                307511 non-null object
CNT_CHILDREN                   307511 non-null int64
AMT_INCOME_TOTAL               307511 non-null float64
AMT_CREDIT                     307511 non-null float64
NAME_INCOME_TYPE               307511 non-null object
NAME_EDUCATION_TYPE            307511 non-null object
NAME_FAMILY_STATUS             307511 non-null object
NAME_HOUSING_TYPE              307511 non-null object
REGION_POPULATION_RELATIVE     307511 non-null float64
DAYS_BIRTH                     307511 non-null int64
DAYS_EMPLOYED                  307511 non-null int64
DAYS_REGISTRATION          

## Clean dataframe --> identify data type --> force data size --> fill missing values

In [28]:
#TASK: BETA: Force data type 
num_stats = {
'flt':{ 'flt64_min':np.finfo(np.float64).min, 'flt64_max':np.finfo(np.float64).max, 'flt32_min':np.finfo(np.float32).min,
'flt32_max':np.finfo(np.float32).max, 'flt16_min':np.finfo(np.float16).min, 'flt16_max':np.finfo(np.float16).max},
'int':
{ 'int64_min':np.iinfo(np.int64).min, 'int64_max':np.iinfo(np.int64).max, 'int32_min':np.iinfo(np.int32).min,
'int32_max':np.iinfo(np.int32).max, 'int16_min':np.iinfo(np.int16).min, 'int16_max':np.iinfo(np.int16).max, 
'int8_min':np.iinfo(np.int8).min, 'int8_min':np.iinfo(np.int8).max }
}

def force_dtype():
    if df_test[each_col].dtype in ['int']:
    #int8 condition already forced in binary column test
    #if col.min() >=  num_stats['int']['int8_min'].min and col.max() <=  num_stats['int']['int8_max'].max:
    #    col.astype(np.int8)
        if df_test[each_col].min() >=  num_stats['int']['int16_min'] and df_test[each_col].max() <=  num_stats['int']['int16_max']:
            df_test[each_col]= df_test[each_col].astype(np.int16)
        elif df_test[each_col].min() >=  num_stats['int']['int32_min'] and df_test[each_col].max() <=  num_stats['int']['int32_max']:
            df_test[each_col]= df_test[each_col].astype(np.int32)
        
        else: 
            #col.min() >=  num_stats['int']['int64_min'].min and col.max() <=  num_stats['int']['int64_max'].max
            df_test[each_col]= df_test[each_col].astype(np.int64)
    else:
    
        if df_test[each_col].min() >=  num_stats['flt']['flt16_min'] and df_test[each_col].max() <=  num_stats['flt']['flt16_max']:
            df_test[each_col]= df_test[each_col].astype(np.float16)
        elif df_test[each_col].min() >=  num_stats['flt']['flt32_min'] and df_test[each_col].max() <=  num_stats['flt']['flt32_max']:
            df_test[each_col] =  df_test[each_col].astype(np.float32)
            
        else: 
            df_test[each_col]= df_test[each_col].astype(np.float64)


all_other_int_col=set()
col_int_bin =set()
col_obj=set()
col_obj_cat=set()
col_flt=set()

for each_col in df_test.columns:
    if df_test[each_col].dtype == 'object':
# Count object columns
        col_obj.add(each_col)
# Fill missing values
        df_test[each_col].fillna('unknown', inplace=True)             
# identify if column is categorical based on number of unique values (5%)
        if df_test[each_col].nunique()/len(df_test[each_col]) <= 0.05:
# encode categorical column
            df_test_temp = pd.get_dummies(df_test[each_col],prefix=each_col)
# add new columns to col_obj_cat, so we can check against full count at end
            for each_new_dummy_col in df_test_temp.columns:
                col_obj_cat.add(each_new_dummy_col)
            
    elif df_test[each_col].dtype == 'float64' or df_test[each_col].dtype == 'float32' or df_test[each_col].dtype == 'float16':
        df_test[each_col].fillna(df_test[each_col].mean(), inplace=True)
        col_flt.add(each_col)
        force_dtype()
    elif df_test[each_col].dtype == 'int64' or df_test[each_col].dtype == 'int32' or df_test[each_col].dtype == 'int16' or df_test[each_col].dtype == 'int8':
# check for binary classification for missing values to -1, and force int8
# if the missing values in a column are binary in nature (0 or 1), we do not want median, we want -1        
        if df_test[each_col].min() == 0 and df_test[each_col].max() == 1 and df_test[each_col].nunique == 2:
            df_test[each_col].fillna(-1, inplace=True)
            col_int_bin.add(each_col)
            force_dtype()
#Else if not binary@int8 then force int16, int32 or int64 and fill missing values with median
        else:
            df_test[each_col].fillna(df_test[each_col].median(), inplace=True)
            all_other_int_col.add(each_col)
            force_dtype()
df_test_new = pd.concat([df_test, df_test_temp], axis = 1)
df_test = pd.concat([df_test, df_test_temp], 1)        


# check missing values again and if all data type seperated missing vaule columns and total missing columns    :
if df_test.isnull().sum().sum() == 0 and len(col_obj)+ len(col_obj_cat) + len(all_other_int_col) + len(col_int_bin) + len(col_flt) == len(df_test.columns):
    print( 'All missing columns seperated by data types filled and accounted for!')
else:
    print("There are errors in processing")
#print(f'object type : {len(col_obj)}, integer type: {len(col_int)}, float type: {len(col_flt)}')   






There are errors in processing


In [29]:
len(df_test.columns)

70

In [484]:
for each_col in df_test.columns:
    if df_test[each_col].dtype == 'object':

# Fill missing values
        df_test[each_col].fillna('unknown', inplace=True)             
# identify if column is categorical based on number of unique values (5%)
        if df_test[each_col].nunique()/len(df_test[each_col]) <= 0.75:
# encode categorical column
            df_test_temp = pd.get_dummies(df_test[each_col],prefix=each_col)
            df_test_new= pd.concat([df_test, df_test_temp], 1)

In [27]:
df_test = df_app_train_missing.copy()

In [1]:
df_test.shape

NameError: name 'df_test' is not defined

In [435]:
#Number of null_values across all columns:
df_test.isnull().sum().sum()

9152465

In [436]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 67 columns):
AMT_ANNUITY                     307499 non-null float64
AMT_GOODS_PRICE                 307233 non-null float64
NAME_TYPE_SUITE                 306219 non-null object
OWN_CAR_AGE                     104582 non-null float64
OCCUPATION_TYPE                 211120 non-null object
CNT_FAM_MEMBERS                 307509 non-null float64
EXT_SOURCE_1                    134133 non-null float64
EXT_SOURCE_2                    306851 non-null float64
EXT_SOURCE_3                    246546 non-null float64
APARTMENTS_AVG                  151450 non-null float64
BASEMENTAREA_AVG                127568 non-null float64
YEARS_BEGINEXPLUATATION_AVG     157504 non-null float64
YEARS_BUILD_AVG                 103023 non-null float64
COMMONAREA_AVG                  92646 non-null float64
ELEVATORS_AVG                   143620 non-null float64
ENTRANCES_AVG                   152683 non-null floa

9152465

In [340]:
all_other_int_col

[]

In [341]:
col_int_bin

[]

In [342]:
col_obj

[]

In [343]:
col_flt

[]

In [417]:
x =set()

In [418]:

x.update([1, 4, 2, 6, 3, 4, 3, 2, 7])

In [419]:
x

{1, 2, 3, 4, 6, 7}

#### WORKING CODE: 
1) all column dtypes split \
2) all data types sized forced  *** NOT DONE*** \
3) all missing values filled \
4) all categorical columns encoded *** NOT DONE*** \
5) all columns accounted for \